In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import random

IMAGE_WIDTH = 224
IMAGE_HEIGHT = 224
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3
BATCH_SIZE = 32
NUM_CLASSES = 1
EPOCHS_HEAD_TRAINING = 15
EPOCHS_FINE_TUNING = 30
LEARNING_RATE_HEAD = 1e-3
LEARNING_RATE_FINE_TUNE = 1e-5
VAL_SPLIT = 0.2

In [ ]:
import zipfile

with zipfile.ZipFile('ads.zip', 'r') as zip_ref:
    zip_ref.extractall('unzipped')


In [ ]:
BASE_DATA_PATH = '/content/unzipped/ads/ads'
TRAIN_DIR = '/content/train'
VAL_DIR = '/content/val'

def create_train_val_dirs(base_path, train_path, val_path, val_split=0.2):
    if os.path.exists(train_path):
        shutil.rmtree(train_path)
    if os.path.exists(val_path):
        shutil.rmtree(val_path)

    os.makedirs(train_path, exist_ok=True)
    os.makedirs(val_path, exist_ok=True)

    for class_name in os.listdir(base_path):
        class_dir = os.path.join(base_path, class_name)
        if os.path.isdir(class_dir):
            os.makedirs(os.path.join(train_path, class_name), exist_ok=True)
            os.makedirs(os.path.join(val_path, class_name), exist_ok=True)

            images = [f for f in os.listdir(class_dir) if os.path.isfile(os.path.join(class_dir, f))]
            random.shuffle(images) # Shuffle for random split

            split_idx = int(len(images) * (1 - val_split))
            train_images = images[:split_idx]
            val_images = images[split_idx:]

            for img_name in train_images:
                shutil.copy(os.path.join(class_dir, img_name), os.path.join(train_path, class_name, img_name))
            for img_name in val_images:
                shutil.copy(os.path.join(class_dir, img_name), os.path.join(val_path, class_name, img_name))
    print(f"Created train and validation directories at {train_path} and {val_path}")

In [ ]:
create_train_val_dirs(BASE_DATA_PATH, TRAIN_DIR, VAL_DIR, val_split=VAL_SPLIT)


Created train and validation directories at /content/train and /content/val


In [ ]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input

# Training data generator with augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,          # Randomly rotate images
    width_shift_range=0.2,      # Randomly shift images horizontally
    height_shift_range=0.2,     # Randomly shift images vertically
    shear_range=0.2,            # Shear Intensity
    zoom_range=0.2,             # Randomly zoom image
    horizontal_flip=True,       # Randomly flip images horizontally
    fill_mode='nearest'         # Strategy for filling newly created pixels
)

# Validation data generator (only rescaling/preprocessing, no augmentation)
validation_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

# Flow data from directories
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary', # For binary classification
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False # No need to shuffle validation data
)

Found 5283 images belonging to 2 classes.
Found 1322 images belonging to 2 classes.


In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False,
                      input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))

# Freeze the layers of the base model initially
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x) # Important to reduce dimensions
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x) # Helps stabilize training
x = Dropout(0.5)(x)         # Regularization
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.3)(x)
predictions = Dense(NUM_CLASSES, activation='sigmoid')(x) # Sigmoid for binary

model = Model(inputs=base_model.input, outputs=predictions)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
print("\n--- Training the classification head ---")
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE_HEAD),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
model_checkpoint_head = ModelCheckpoint('best_model_head.keras', save_best_only=True, monitor='val_loss') # .keras extension
reduce_lr_head = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

history_head = model.fit(
    train_generator,
    #steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS_HEAD_TRAINING,
    validation_data=validation_generator,
    #validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping, model_checkpoint_head, reduce_lr_head]
)



--- Training the classification head ---
Epoch 1/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 108s 570ms/step - accuracy: 0.8502 - loss: 0.3395 - val_accuracy: 0.8530 - val_loss: 0.3490 - learning_rate: 0.0010
Epoch 2/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 85s 514ms/step - accuracy: 0.8591 - loss: 0.3404 - val_accuracy: 0.8568 - val_loss: 0.3200 - learning_rate: 0.0010
Epoch 3/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 87s 526ms/step - accuracy: 0.8544 - loss: 0.3336 - val_accuracy: 0.8644 - val_loss: 0.3272 - learning_rate: 0.0010
Epoch 4/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 85s 512ms/step - accuracy: 0.8597 - loss: 0.3243 - val_accuracy: 0.8682 - val_loss: 0.3077 - learning_rate: 0.0010
Epoch 5/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 83s 499ms/step - accuracy: 0.8640 - loss: 0.3117 - val_accuracy: 0.8652 - val_loss: 0.3114 - learning_rate: 0.0010
Epoch 6/15
166/166 ━━━━━━━━━━━━━━━━━━━━ 83s 503ms/step - accuracy: 0.8731 - loss: 0.3004 - val_accuracy: 0.8652 - val_loss: 0.3131 - learning_rate: 0.0010
Epoch 7/15
166/166 ━━━━━━━━

In [ ]:
for layer in base_model.layers:
    layer.trainable = True
print(f"Number of layers in base_model: {len(base_model.layers)}")

Number of layers in base_model: 175


In [ ]:
model.compile(optimizer=Adam(learning_rate=LEARNING_RATE_FINE_TUNE), # Crucial: very low LR
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Callbacks for fine-tuning
model_checkpoint_fine_tune = ModelCheckpoint('best_model_fine_tuned.keras', save_best_only=True, monitor='val_loss') # .keras extension
reduce_lr_fine_tune = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-7, verbose=1)
# Early stopping can be more aggressive here or reuse the previous one if patience is suitable.
early_stopping_ft = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True, verbose=1)


history_fine_tune = model.fit(
    train_generator,
    #steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS_FINE_TUNING,
    validation_data=validation_generator,
    #validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[early_stopping_ft, model_checkpoint_fine_tune, reduce_lr_fine_tune],
    initial_epoch=history_head.epoch[-1] +1 if hasattr(history_head, 'epoch') and history_head.epoch else 0 # Continue epoch count
)

Epoch 14/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 195s 777ms/step - accuracy: 0.8371 - loss: 0.3705 - val_accuracy: 0.8735 - val_loss: 0.3225 - learning_rate: 1.0000e-05
Epoch 15/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 103s 616ms/step - accuracy: 0.8708 - loss: 0.3044 - val_accuracy: 0.8682 - val_loss: 0.3179 - learning_rate: 1.0000e-05
Epoch 16/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 109s 653ms/step - accuracy: 0.8825 - loss: 0.2816 - val_accuracy: 0.8742 - val_loss: 0.3150 - learning_rate: 1.0000e-05
Epoch 17/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 107s 641ms/step - accuracy: 0.8876 - loss: 0.2648 - val_accuracy: 0.8750 - val_loss: 0.3132 - learning_rate: 1.0000e-05
Epoch 18/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 117s 702ms/step - accuracy: 0.8876 - loss: 0.2542 - val_accuracy: 0.8765 - val_loss: 0.3130 - learning_rate: 1.0000e-05
Epoch 19/30
166/166 ━━━━━━━━━━━━━━━━━━━━ 107s 647ms/step - accuracy: 0.8956 - loss: 0.2367 - val_accuracy: 0.8780 - val_loss: 0.3020 - learning_rate: 1.0000e-05
Epoch 20/30
166/166 ━━━━━━━━━━━━━━

In [ ]:
def plot_history(history_head, history_fine_tune=None, initial_epochs=0):
    acc = history_head.history['accuracy']
    val_acc = history_head.history['val_accuracy']
    loss = history_head.history['loss']
    val_loss = history_head.history['val_loss']

    if history_fine_tune:
        acc += history_fine_tune.history['accuracy']
        val_acc += history_fine_tune.history['val_accuracy']
        loss += history_fine_tune.history['loss']
        val_loss += history_fine_tune.history['val_loss']

    epochs_range_head = range(len(history_head.history['accuracy']))
    total_epochs = len(acc)
    epochs_range_total = range(total_epochs)


    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(epochs_range_total, acc, label='Training Accuracy')
    plt.plot(epochs_range_total, val_acc, label='Validation Accuracy')
    if history_fine_tune:
        plt.plot([len(epochs_range_head)-1, len(epochs_range_head)-1],
                 plt.ylim(), label='Start Fine-Tuning', linestyle='--')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')


    plt.subplot(1, 2, 2)
    plt.plot(epochs_range_total, loss, label='Training Loss')
    plt.plot(epochs_range_total, val_loss, label='Validation Loss')
    if history_fine_tune:
         plt.plot([len(epochs_range_head)-1, len(epochs_range_head)-1],
                 plt.ylim(), label='Start Fine-Tuning', linestyle='--')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')

    plt.tight_layout()
    plt.show()

plot_history(history_head, history_fine_tune, initial_epochs=EPOCHS_HEAD_TRAINING)


In [ ]:
print("\nLoading best fine-tuned model for evaluation...")
best_model = tf.keras.models.load_model('best_model_fine_tuned.keras')

# Evaluate on validation set (or a separate test set if you have one)
eval_results = best_model.evaluate(validation_generator,
                                   steps=validation_generator.samples // BATCH_SIZE,
                                   verbose=1)
print(f"\nBest Fine-tuned Model Validation Loss: {eval_results[0]:.4f}")
print(f"Best Fine-tuned Model Validation Accuracy: {eval_results[1]:.4f}")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

Y_pred_probs = best_model.predict(validation_generator, steps=validation_generator.samples // BATCH_SIZE +1) # Ensure all samples are predicted
Y_pred = (Y_pred_probs > 0.5).astype(int) # Threshold at 0.5 for binary
y_true = validation_generator.classes # True labels

num_val_samples = validation_generator.samples
if len(Y_pred) > num_val_samples:
    Y_pred = Y_pred[:num_val_samples]

print('\nConfusion Matrix')
cm = confusion_matrix(y_true, Y_pred)
print(cm)

print('\nClassification Report')

class_labels = list(validation_generator.class_indices.keys())
print(classification_report(y_true, Y_pred, target_names=class_labels))